# Onboard API and API Wrapper: Introduction
In this notebook we will be exploring the Onboard API and API wrapper. Make sure to run the following chunk of code first in order to install it.

In [ ]:
# install API wrapper first

!pip install onboard.client

In [ ]:
import pandas as pd
from onboard.client import RtemClient

# What is an API?

API stands for **Application Programming Interface** and the definition is kind of loose. Let's check out two of them to understand what they do. First, the [Wikipedia](https://en.wikipedia.org/wiki/API) definiton:

> *An API is a connection between computers or between computer programs. It is a type of software interface, offering a service to other pieces of software. A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.*

Simple but confusing, right? Fortunately [this reddit user has a simpler and more understandable definition](https://www.reddit.com/r/explainlikeimfive/comments/lagk1/eli5_what_is_an_api/c2r8hie/):

> *Imagine you have a little toy car. You can move that toy car forward and backwards with your hands, and make it go slow or fast, and do all sorts of other things that car toys can do. But then you decide you want your friend, Jimmy, to be able to control the car as well. But you don't want to give the car to him outright -- it's your car! You just want to let him control it without needing to move it for him. So you build a remote control for the car and give it to Jimmy. That remote control has a specific set of functions that tell the car what to do: go forward, backwards, and so on. Since it's your car, you programmed the remote and gave it limitations so Jimmy could only do certain things with it. Jimmy takes the remote and controls the car and has fun, but can't do anything with it beyond what the remote control will let him. An API is the remote control. The car is an application.*

Essentialy, an API is an interface that let's you use a complex software in an easier and controlled way. Here we will using what is called a [REST](https://en.wikipedia.org/wiki/Representational_state_transfer) API; you don't need to get deep into it to use it, just know that they exists and have the link in case you want to dive into it. What you need to have in mind is:

- The client (you) make **requests** to the server through the API
- The server returns a **response** to the cliente through the API

And in those requests there are different actions that the API can perform:

- **GET** retrieves data from an API.
- **POST** sends new data to an API.
- **PATCH** and **PUT** update existing data.
- **DELETE** removes existing data.

All these actions are performed to an specific **endpoint**: these are defined urls to performed pre-defined actions. For example, you get your building's data from and endpoint and you get your users data from another one.

# How to get access?

Now that we know how an API works let's focus on the Onboard API. First thing you need is to get access, you can not simply start making requests ;)

You will need an API key, let's review some definitions before starting.

## API Keys

As a general definition, API keys are personal tokens commonly used to control the utilization of the API’s interface and track how it is being used. This is often as a precaution to prevent abuse or malicious use. 

In the particular case of Onboard API, API keys allow secure, automated access to the Onboard portal. They are granted explicit access to API endpoints and data by attaching one or more scopes to the key. The key is linked to its creator's user account, and can only access information that is already visible in the UI. This means that an API key which has been granted all scopes is equivalent to a logged in user.

So to start using Onboard API, **make sure you not only have an account, but you also have generated an API key**. In the following section is explained how to do it.

## Generating API keys

 In the [official documentation you can find the instructions to generate your API Key](https://onboard-data-python-client-api.readthedocs.io/en/latest/Initial%20Setup.html#setting-up-api-access):

> *If you are an existing Onboard user you can head over to the accounts page and generate a new key and grant scopes for “general” and “buildings:read”.
If you would like to get access to Onboard and start prototyping against an example building please request [access here](https://www.onboarddata.io/contact-us).
If you’re participating in the [NYSERDA hackathon](https://www.rtemhackathon.com/), please [click here](https://www.rtemhackathon.com/) to get set up with API access for the competition.*

Once you have created it you can come back to this notebook and check if it's working, use your API key with access to the **general scope** and **buiding:read scope**:

In [ ]:
api_key = 'ob-p-dUtFibU1B4nEyTiUM2oWxCQwP8WSh5PKcaQ6cPclEed09OqogBJuwnVb5PDszjk93rc'

In [ ]:
# You can ignore this part, is for keeping my API key secret
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("hackaton_key")

In [ ]:
client = RtemClient(api_key=api_key)

In [ ]:
client.whoami()

# Using API end-points vs Onboard API wrapper

## API

We will make some **requests** to the API using the library *requests* (how original) to perform them from this notebook.

In [ ]:
import requests

Before anything, we need to authenticate with our API key:

In [ ]:
# This is the API key
key = { "key": api_key}

# This is a POST request
response = requests.post("https://api.onboarddata.io/login/api-key", data=key)

# This is the response in json format
response = response.json()

And we got a response! but what is it? what's a json? [Here is a nice definition](https://www.copterlabs.com/json-what-it-is-how-it-works-how-to-use-it/):

> *JSON is short for JavaScript Object Notation, and is a way to store information in an organized, easy-to-access manner. In a nutshell, it gives us a human-readable collection of data that we can access in a really logical manner.*

Typically, web services (like this API) returns a json when making a request. Let's check the json response we got:

In [ ]:
response

Looks like a dictionary, righ? Well...

In [ ]:
type(response)

It is a dictionary! This is because python supports json and the *requests* library parse it automatically using the method `.json()`.

Well, now is time to GET some information from our API; we will need our login response, the `acces_token` it retuned in the json.

In [ ]:
response["access_token"]

But we need it in a certain way, we need it as a header: request headers are extra information to use during the request, in this case, we need to give the acces token to authenticate ourselves and get a response. Onboard API needs the following header:

In [ ]:
headers = {"Authorization": "Bearer "+ response["access_token"]}
headers

And finally we can make a request! Let's request the buildings in our data trough the buildings **endpoint**: remember that this is an specific url for that purpose (you can check all the endpoints in the [API documentation from your account](https://portal.onboarddata.io/docs/swagger)):

- GET request
- endpoint: https://api.onboarddata.io/buildings
- Authorization header

Using the *requests* library:

In [ ]:
bdgs = requests.get("https://api.onboarddata.io/buildings", headers=headers).json()
# Check only the first one
bdgs[0]

And we have a json (dict) with our buildings! We can convert it to a DataFrame using `json_normalize` or `pd.DataFrame`, both from *pandas* library:

In [ ]:
pd.json_normalize(bdgs)

In [ ]:
pd.DataFrame(bdgs)

## Onboard API Wrapper

Well, that was a lot of work for a 229 rows DataFrame. But... what if I tell you there is a simpler way to query this data? There is the Onboard API wrapper! which is esentially a library you can import and use here (also called "SDK" that stands for Software Development Kit). We are using the `client` we have previously created.

And with a single line of code we can get our buildings:

In [ ]:
bdgs = client.get_all_buildings()
# Check only the first one
bdgs[0]

In [ ]:
bdgs_df = pd.DataFrame(bdgs)

In [ ]:
bdgs_df

In [ ]:
bdgs_df.info()

Way more simpler than making requests to the API! You can check the API wrapper documentation [here](https://onboard-data-python-client-api.readthedocs.io/en/latest/index.html).

let's explore some more feautures:

In [ ]:
# We can get all the equipment from one of our buidings, let's say the id = 441
all_equipment = pd.DataFrame(client.get_building_equipment(441))

all_equipment.head()

In [ ]:
all_equipment.info()

In this data frame we have listed all the equipment in the select building. Check out the column `points`: these are all the data points asociated to that equipment. You can [query specific points](https://onboard-data-python-client-api.readthedocs.io/en/latest/Querying%20Building-Specific%20Data.html#querying-specific-points) with certain conditions using *PointSelector* (we'll get deeper into this in the following notebook).

In [ ]:
from onboard.client.models import PointSelector

First create your *PointSelector* object:

In [ ]:
query = PointSelector()

And you can specify all the conditions you want ([check the doc](https://onboard-data-python-client-api.readthedocs.io/en/latest/Querying%20Building-Specific%20Data.html#querying-specific-points)):

In [ ]:
query.point_types = ['Zone Temperature']
query.equipment_types = ['fcu']
query.buildings = [417, 162, 259]

And when you execute that query using `select_points` you will get the points that satisfy those conditions:

In [ ]:
selection = client.select_points(query)
selection

An those are the points, identified by id:

In [ ]:
points = selection["points"]
points

And you can get all the details about those points using `get_points_by_ids`:

In [ ]:
sensor_metadata = client.get_points_by_ids(points)
pd.DataFrame(sensor_metadata)

Let's stop for a moment on the `last_updated` and `first_updated`. Looks weird for a datetime, right? This is becauses is in [Unix Time](https://en.wikipedia.org/wiki/Unix_time), those are the miliseconds that passed since Unix Epoch; The Unix epoch is 00:00:00 UTC on 1 January 1970 (an arbitrary date). Fortunately, is really easy to convert those miliseconds to UTC:

In [ ]:
from datetime import datetime, timezone

In [ ]:
# The starting point of Unix Time
datetime.fromtimestamp(0, timezone.utc)

Let's do it for each value in `last_updated`, we can use [`apply` method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) for that (remember that we are converting miliseconds):

In [ ]:
sensor_metadata = pd.DataFrame(sensor_metadata)
sensor_metadata.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc))

Let's inspect the datetime period we are working with; we have to convert the `first_updated` and `last_updated` from unix timestamp to a datetime, like we just did, to find the limits.

In [ ]:
sensor_metadata.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min()

In [ ]:
sensor_metadata.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).max()

We have 2018-2021 data, we are using that information soon when getting the time-series. Because [querying time-series data](https://onboard-data-python-client-api.readthedocs.io/en/latest/Querying%20Building-Specific%20Data.html#querying-time-series-data) is easy too! Let's import the needed libraries first:

In [ ]:
import pytz
from onboard.client.models import TimeseriesQuery, PointData
from onboard.client.dataframes import points_df_from_streaming_timeseries

First we have to select our time period in UTC. As we see before, we have 2018-2021 data. Let's select a period of data; we are using the [python library datetime](https://docs.python.org/3/library/datetime.html) to create datetime objects. Remember that all the data from the API is in UTC, you have to localize all your datetimes using [PyTZ](https://pythonhosted.org/pytz/) a library designed for that purpose.

In [ ]:
# Select your timezone
tz = pytz.timezone('UTC')

# Select your date and time (time is optional)
start = datetime(2018,1,20,0,0,0).replace(tzinfo=tz)
end = datetime(2018,2,20,0,0,0).replace(tzinfo=tz)

print(f"from {start} to {end}")

You can also set relative dates using `timedelta`:

```
start = datetime.now(pytz.utc) - timedelta(days=7)
end = datetime.now(pytz.utc)
```

And let's get the time-series data from the previously selecte `points`. Using `TimeseriesQuery` qe create the query we want to execute:

In [ ]:
timeseries_query = TimeseriesQuery(point_ids = points, start = start, end = end)

And now we execute it and transform to dataframe:

In [ ]:
# Exceute query (will return and object)
query_results = client.stream_point_timeseries(timeseries_query)

# Convert to dataframe
sensor_data = points_df_from_streaming_timeseries(query_results)

In [ ]:
sensor_data.head()

And we have the time-series for our selected points. You can export it as csv:

In [ ]:
#sensor_data.to_csv("sensor_data.csv", index=False)

Finally, let's visualize our sensors data! (we will get a bit deeper on this subject in following notebooks)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# This is for the visual stype, I like "ggplot"
plt.style.use('ggplot')

# This for the figure size
plt.rcParams["figure.figsize"] = (20,9)

In [ ]:
sns.lineplot(data=sensor_data)